In [1]:
import numpy as np
import pandas as pd

In [10]:
years = [2020,2021,2022,2023]
conferences = ['ACC', 'Big10', 'Big12', 'SEC', 'PAC12']

for year in years:
    for con in conferences:

        #Player Stats by Season
        p_stats_season = pd.read_csv(f'Player Stats/Player Stats By Season {con} {year}.csv')
        #Remove 'Team' Player
        p_stats_season = p_stats_season[p_stats_season['Player'] != ' Team']

        #Group Player Stats by PlayerId
        p_stats_season = p_stats_season[['PlayerId', 'StatType', 'Stat']].groupby(['PlayerId', 'StatType']).sum().unstack()
        p_stats_season.columns = p_stats_season.columns.droplevel(0)
        p_stats_season.reset_index(inplace = True)
        p_stats_season = p_stats_season.rename_axis(None, axis=1)
        p_stats_season.index = p_stats_season.index.map(int)

        #Player usage data
        p_usage = pd.read_csv(f'Player Usage/Player Usage {con} {year}.csv')
        p_usage = p_usage.rename(columns = {'Name':'Player', 'Id': 'PlayerId'}).sort_values(by = 'Player').reset_index(drop = True)

        #Merge Player Stats and Player Usage
        merged = p_usage.merge(p_stats_season, on = 'PlayerId', how = 'outer')
        merged['Season'] = merged['Season'].apply(lambda x: 2023)
        merged['Conference'] = merged['Conference'].apply(lambda x: 'Big Ten')


        #2023 ALL CFB Player Basic Info
        p_info = pd.read_csv(f'Player Basic Info/{year} Players Basic Info.csv')
        p_info['Player'] = p_info['First Name'] + ' '+ p_info['Last Name']
        p_info.rename(columns = {'Id': 'PlayerId'}, inplace=True)
        p_info.drop(columns = ['First Name', 'Last Name'], inplace = True)
        p_info.sort_values(by = 'Player', inplace = True)
        p_info.dropna(inplace=True)

        #Team Records
        t_records = pd.read_csv(f'Team Records/Team Records {con} {year}.csv')
        t_records.drop(columns=['Year', 'Conference'], inplace = True)

        #Team Basic Stats
        t_b_stats = pd.read_csv(f'Team Stats/Team Basic Stats {con} {year}.csv')
        t_b_stats = t_b_stats[['Team', 'StatName', 'StatValue']].groupby(['Team', 'StatName']).sum().unstack()
        t_b_stats.columns = t_b_stats.columns.droplevel(0)
        t_b_stats.reset_index(inplace = True)
        t_b_stats = t_b_stats.rename_axis(None, axis=1)
        t_b_stats = t_b_stats.add_prefix('Team ')
        t_b_stats.rename(columns={'Team Team': 'Team'},  inplace=True)

        #Merge to fill in missing player info
        merged2 = merged.merge(p_info, on = 'PlayerId', how = 'left', suffixes=[None, '_y'])
        merged2['Position'] = merged2['Position'].combine_first(merged2['Position_y'])
        merged2['Team'] = merged2['Team'].combine_first(merged2['Team_y'])
        merged2['Player'] = merged2['Player'].combine_first(merged2['Player_y'])
        merged2.drop(columns=['Team_y', 'Position_y', 'Player_y'], inplace= True)
        merged2.sort_values(by = 'PlayerId')
        merged2.dropna(subset = ['Player'], inplace=True)

        #Merge Team Record Info
        merged3 = merged2.merge(t_records, on = 'Team')

        #Merge team basic stats
        merged4 = merged3.merge(t_b_stats, on = 'Team')

        #Load transfer data 
        transfer = pd.read_csv(f'Transfer Data/{year+1} Transfer.csv')
        transfer['Player'] = transfer['FirstName'] + ' ' + transfer['LastName']
        transfer.rename(columns = {'Origin': 'Team'}, inplace = True)
        transfer = transfer[['Player','Team', 'Position', 'Stars']]
        transfer = transfer.assign(Transfer_Portal ='Yes')

        #Merge Data with Transfer Data 
        final_merged = merged4.merge(transfer, on = ['Player', 'Team', 'Position'], how = 'left')
        final_merged['Transfer_Portal'] = final_merged['Transfer_Portal'].fillna('No')

        final_merged.to_csv(f'Merged Datasets Con Year/{con} {year}.csv')

In [11]:
for con in conferences:
    #take in each year
    set1 = pd.read_csv(f'Merged Datasets Con Year/{con} 2020.csv')
    set2 = pd.read_csv(f'Merged Datasets Con Year/{con} 2021.csv')
    set3 = pd.read_csv(f'Merged Datasets Con Year/{con} 2022.csv')
    set4 = pd.read_csv(f'Merged Datasets Con Year/{con} 2023.csv')

    #merge each year
    merged = pd.concat([set1, set2, set3, set4])

    #export to new single conference dataset
    merged.to_csv(f'Merged Datasets Con/{con}.csv')

In [22]:
acc = pd.read_csv(f'Merged Datasets Con/ACC.csv')
big10 = pd.read_csv(f'Merged Datasets Con/Big10.csv')
big12 = pd.read_csv(f'Merged Datasets Con/Big12.csv')
sec = pd.read_csv(f'Merged Datasets Con/SEC.csv')
pac12 = pd.read_csv(f'Merged Datasets Con/PAC12.csv')

final_concat = pd.concat([acc, big10, big12, sec, pac12])

final_concat = final_concat.drop(columns=['Unnamed: 0.1','Unnamed: 0'])

#There was a mishap somewhere in the data downloading, and I need to define the columns we're going to include
cols = ['Season', 'PlayerId', 'Player', 'Position', 'Team', 'Conference',
       'Usage Overall', 'Usage Pass', 'Usage Rush', 'Usage FirstDown',
       'Usage SecondDown', 'Usage ThirdDown', 'Usage StandardDowns',
       'Usage PassingDowns', 'ATT', 'AVG', 'CAR', 'COMPLETIONS', 'FGA', 'FGM',
       'FUM', 'INT', 'In 20', 'LONG', 'LOST', 'NO', 'PCT', 'PD', 'PTS',
       'QB HUR', 'REC', 'SACKS', 'SOLO', 'TB', 'TD', 'TFL', 'TOT', 'XPA',
       'XPM', 'YDS', 'YPA', 'YPC', 'YPP', 'YPR', 'Division', 'ExpectedWins',
       'Total Games', 'Total Wins', 'Total Losses', 'Total Ties',
       'ConferenceGames Games', 'ConferenceGames Wins',
       'ConferenceGames Losses', 'ConferenceGames Ties', 'HomeGames Games',
       'HomeGames Wins', 'HomeGames Losses', 'HomeGames Ties',
       'AwayGames Games', 'AwayGames Wins', 'AwayGames Losses',
       'AwayGames Ties', 'Team firstDowns', 'Team fourthDownConversions',
       'Team fourthDowns', 'Team fumblesLost', 'Team fumblesRecovered',
       'Team games', 'Team interceptionTDs', 'Team interceptionYards',
       'Team interceptions', 'Team kickReturnTDs', 'Team kickReturnYards',
       'Team kickReturns', 'Team netPassingYards', 'Team passAttempts',
       'Team passCompletions', 'Team passesIntercepted', 'Team passingTDs',
       'Team penalties', 'Team penaltyYards', 'Team possessionTime',
       'Team puntReturnTDs', 'Team puntReturnYards', 'Team puntReturns',
       'Team rushingAttempts', 'Team rushingTDs', 'Team rushingYards',
       'Team sacks', 'Team tacklesForLoss', 'Team thirdDownConversions',
       'Team thirdDowns', 'Team totalYards', 'Team turnovers', 'Stars',
       'Transfer_Portal']

final_concat = final_concat[cols]

final_concat.to_csv('Final Dataset.csv', index=False)

In [26]:
data = pd.read_csv("Final Dataset.csv")

data

,Season,PlayerId,Player,Position,Team,Conference,Usage Overall,Usage Pass,Usage Rush,Usage FirstDown,...,Team rushingTDs,Team rushingYards,Team sacks,Team tacklesForLoss,Team thirdDownConversions,Team thirdDowns,Team totalYards,Team turnovers,Stars,Transfer_Portal
0,2023,548077,Ryan Smith,LB,Duke,Big Ten,NaN,NaN,NaN,NaN,...,19,1724,31,67,65,168,4165,39,NaN,No
1,2023,3121655,Justus Reed,DL,Virginia Tech,Big Ten,NaN,NaN,NaN,NaN,...,27,2648,36,75,51,133,4860,14,NaN,No
2,2023,3858274,Luc Bequette,DL,Boston College,Big Ten,NaN,NaN,NaN,NaN,...,11,1119,24,52,65,147,4245,14,3.0,Yes
3,2023,3895797,Tre Tipton,WR,Pittsburgh,Big Ten,0.0186,0.0343,0.0,0.0202,...,20,1343,45,110,63,173,4200,16,NaN,No
4,2023,3914537,Miles Fox,DL,Wake Forest,Big Ten,NaN,NaN,NaN,NaN,...,22,1491,16,68,56,139,4024,7,NaN,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12851,2023,5159802,Dallen Bentley,TE,Utah,Big Ten,0.0011,0.0029,0.0,0.0000,...,19,2373,36,75,81,196,4529,13,NaN,No
12852,2023,5159817,Joseph McGinnis II,DB,Arizona State,Big Ten,NaN,NaN,NaN,NaN,...,17,1334,26,65,53,173,3858,20,NaN,No
12853,2023,5161140,Ryan McCulloch,LB,California,Big Ten,NaN,NaN,NaN,NaN,...,26,2250,25,58,74,191,5086,28,NaN,No
12854,2023,5161141,Mateen Bhaghani,PK,California,Big Ten,NaN,NaN,NaN,NaN,...,26,2250,25,58,74,191,5086,28,NaN,No
